## 프로젝트 : 커스텀 프로젝트 직접 만들기

### 라이브러리 버전을 확인해 봅니다.

In [1]:
import tensorflow
import numpy
import transformers
import datasets

print(tensorflow.__version__)
print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

2.6.0
1.21.4
4.11.3
1.14.0


### STEP 1. NSMC 데이터 분석 및 Huggingface dataset 구성


In [2]:
import datasets
from datasets import load_dataset

dataset = load_dataset("nsmc")

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [4]:
train = dataset['train']
cols = train.column_names
cols

['id', 'document', 'label']

In [5]:
for i in range(5):
    for col in cols:
        print(col, ":", train[col][i])
    print('\n')

id : 9976970
document : 아 더빙.. 진짜 짜증나네요 목소리
label : 0


id : 3819312
document : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
label : 1


id : 10265843
document : 너무재밓었다그래서보는것을추천한다
label : 0


id : 9045019
document : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
label : 0


id : 6483659
document : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
label : 1




### STEP 2. klue/bert-base model 및 tokenizer 불러오기

In [38]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
huggingface_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/klue/bert-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/1a36e69d48a0

In [39]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )


In [40]:
hf_dataset = dataset.map(transform, batched=True)
hf_test_dataset = dataset.map(transform, batched=True)

# train & validation & test split
hf_train_dataset = hf_dataset['train'].shuffle(seed=42).select(range(5000))
hf_val_dataset = hf_dataset['test'].shuffle(seed=42).select(range(300))
hf_test_dataset = hf_test_dataset['test'].shuffle(seed=42).select(range(500))

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5e17ed5be458f579.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-f0a2f9b1eed1bc40.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5e17ed5be458f579.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-f0a2f9b1eed1bc40.arrow
Loading cached shuffled indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-419bde005ee4aa8f.arrow
Loading cached shuffled indices for dataset at /aiffel/.cache/hu

In [41]:
print(hf_train_dataset.shape)
print(hf_test_dataset.shape)

(5000, 5)
(500, 5)


### STEP 3. 위에서 불러온 tokenizer으로 데이터셋을 전처리하고, model 학습 진행해 보기

In [10]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 1,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

In [11]:
from datasets import load_metric

#metric = load_metric('glue', 'mnli')
metric = load_metric("accuracy")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [12]:
# 훈련
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()


The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running training *****
  Num examples = 1000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 125


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.386575,0.843333


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, document.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=125, training_loss=0.5025383911132812, metrics={'train_runtime': 100.6321, 'train_samples_per_second': 9.937, 'train_steps_per_second': 1.242, 'total_flos': 263111055360000.0, 'train_loss': 0.5025383911132812, 'epoch': 1.0})

### STEP 4. Fine-tuning을 통하여 모델 성능(accuarcy) 향상시키기

In [33]:
import torch
torch.cuda.empty_cache()

In [42]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 5,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
from datasets import load_metric

#metric = load_metric('glue', 'mnli')
metric = load_metric("accuracy")

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references = labels)

In [44]:
# 훈련
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 5000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3125


Epoch,Training Loss,Validation Loss,Accuracy
1,0.405700,0.355078,0.860000
2,0.264300,0.609889,0.846667
3,0.210500,0.743493,0.853333
4,0.066700,0.802225,0.860000
5,0.040200,0.854404,0.856667


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Co

TrainOutput(global_step=3125, training_loss=0.1793453594970703, metrics={'train_runtime': 2552.3442, 'train_samples_per_second': 9.795, 'train_steps_per_second': 1.224, 'total_flos': 6577776384000000.0, 'train_loss': 0.1793453594970703, 'epoch': 5.0})

In [45]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 0.9222767353057861,
 'eval_accuracy': 0.85,
 'eval_runtime': 18.1107,
 'eval_samples_per_second': 27.608,
 'eval_steps_per_second': 3.479,
 'epoch': 5.0}

In [46]:
del huggingface_model

### STEP 5. Bucketing을 적용하여 학습시키고, STEP 4의 결과와의 비교


In [47]:
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

huggingface_tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
huggingface_model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

loading configuration file https://huggingface.co/klue/bert-base/resolve/main/config.json from cache at /aiffel/.cache/huggingface/transformers/fbd0b2ef898c4653902683fea8cc0dd99bf43f0e082645b913cda3b92429d1bb.99b3298ed554f2ad731c27cdb11a6215f39b90bc845ff5ce709bb4e74ba45621
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading file https://huggingface.co/klue/bert-base/resolve/main/vocab.txt from cache at /aiffel/.cache/huggingface/transformers/1a36e69d48a0

In [48]:
def transform(data):
    return huggingface_tokenizer(
        data['document'],
        truncation = True,
        padding = 'max_length',
        return_token_type_ids = False,
        )


In [49]:
hf_dataset = dataset.map(transform, batched=True)
hf_test_dataset = dataset.map(transform, batched=True)

# train & validation & test split
hf_train_dataset = hf_dataset['train'].shuffle(seed=42).select(range(5000))
hf_val_dataset = hf_dataset['test'].shuffle(seed=42).select(range(300))
hf_test_dataset = hf_test_dataset['test'].shuffle(seed=42).select(range(500))

Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5e17ed5be458f579.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-f0a2f9b1eed1bc40.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-5e17ed5be458f579.arrow
Loading cached processed dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-f0a2f9b1eed1bc40.arrow
Loading cached shuffled indices for dataset at /aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3/cache-419bde005ee4aa8f.arrow
Loading cached shuffled indices for dataset at /aiffel/.cache/hu

In [50]:
print(hf_train_dataset.shape)
print(hf_test_dataset.shape)

(5000, 5)
(500, 5)


In [51]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=huggingface_tokenizer)

In [54]:
import os
import numpy as np
from transformers import Trainer, TrainingArguments

output_dir = os.getenv('HOME')+'/aiffel/transformers'

training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    evaluation_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 8,   # 각 device 당 batch size
    per_device_eval_batch_size = 8,    # evaluation 시에 batch size
    num_train_epochs = 5,                     # train 시킬 총 epochs
    weight_decay = 0.01,                          # weight decay
    group_by_length = True,
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [55]:
# 훈련
trainer = Trainer(
    model=huggingface_model,           # 학습시킬 model
    args=training_arguments,           # TrainingArguments을 통해 설정한 arguments
    train_dataset=hf_train_dataset,    # training dataset
    eval_dataset=hf_val_dataset,       # evaluation dataset
    compute_metrics=compute_metrics,
    # datacollector
    data_collator=data_collator
    
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 5000
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3125


Epoch,Training Loss,Validation Loss,Accuracy
1,0.268400,0.397895,0.846667
2,0.248400,0.640191,0.860000
3,0.160500,0.802147,0.870000
4,0.049300,0.889541,0.856667
5,0.025400,0.971613,0.866667


Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-500
Configuration saved in /aiffel/aiffel/transformers/checkpoint-500/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1000
Configuration saved in /aiffel/aiffel/transformers/checkpoint-1000/config.json
Model weights saved in /aiffel/aiffel/transformers/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
Saving model checkpoint to /aiffel/aiffel/transformers/checkpoint-1500
Co

TrainOutput(global_step=3125, training_loss=0.13596177200317383, metrics={'train_runtime': 2560.0843, 'train_samples_per_second': 9.765, 'train_steps_per_second': 1.221, 'total_flos': 6577776384000000.0, 'train_loss': 0.13596177200317383, 'epoch': 5.0})

In [56]:
trainer.evaluate(hf_test_dataset)

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8


{'eval_loss': 1.0854955911636353,
 'eval_accuracy': 0.848,
 'eval_runtime': 18.3946,
 'eval_samples_per_second': 27.182,
 'eval_steps_per_second': 3.425,
 'epoch': 5.0}

### before bucketing 
TrainOutput(global_step=3125, training_loss=0.1793453594970703, metrics={'train_runtime': 2552.3442, 'train_samples_per_second': 9.795, 'train_steps_per_second': 1.224, 'total_flos': 6577776384000000.0, 'train_loss': 0.1793453594970703, 'epoch': 5.0})


### After bucketing 
TrainOutput(global_step=3125, training_loss=0.13596177200317383, metrics={'train_runtime': 2560.0843, 'train_samples_per_second': 9.765, 'train_steps_per_second': 1.221, 'total_flos': 6577776384000000.0, 'train_loss': 0.13596177200317383, 'epoch': 5.0})

bucketing을 적용한 결과 대체적으로 train 시간이 더 느려지고 loss도 더 늦게 떨어졌다. 그렇지만 성능은 Bucketing을 적용한 것이 더 높게 나타났다

### 회고
- 데이터를 늘려서 해야 성능이 좋을 것 같은데 GPU 메모리가 부족하다고 해서 fine-tuning의 한계가 있었다.
- hugging face를 사용해보는 의미있는 시간이었다.
- 매우 간단하게 구성되어 있어 새로운 모델을 꼭 만들어야하는 상황이 아니라면 자신의 task에 맞는 pre-trained 모델을 가져다가 조금만 변형해서 쓰는 것이 훨씬 실용적일 수도 있겠다는 생각이들었다.. 